In [2]:
import csv
import os 
from tqdm import tqdm
from operator import itemgetter
import json


In [3]:
def parse_ray_csv_to_dict(filename: str): 
    with open(filename) as csvfile:
        reader = csv.reader(csvfile,delimiter=",")
        keys = reader.__next__()
        path_dict = {}
        timestamp_dict = {}
        #these are the pathIds that have been bagged but have not yet been unbagged
        for row in reader:
#             print(row)
            curr_pathId = int(row[1])
            curr_hop = int(row[2])
            curr_treeletId = int(row[3])
            curr_action = row[4]
            curr_timestamp = int(row[5])
            if curr_action != "U":
                continue;
            else:
                if curr_pathId not in path_dict:
                    path_dict[curr_pathId] = [{"hop": curr_hop,
                                                   "treeletId": curr_treeletId,
                                                    "action": curr_action,
                                                    "timestamp": curr_timestamp}]
                else:
                    path_dict[curr_pathId].append({"hop": curr_hop,
                                                   "treeletId": curr_treeletId,
                                                    "action": curr_action,
                                                    "timestamp": curr_timestamp})
        for elem in path_dict.keys():
            path_dict[elem] = sorted(path_dict[elem], key=itemgetter('hop'))
            sorted_transfers = []
            for idx in range(1,len(path_dict[elem])):
                sorted_transfers.append({"src_dst": [path_dict[elem][idx-1]['treeletId'],
                                                         path_dict[elem][idx]['treeletId']],
                                             "timestamp":int(path_dict[elem][idx]['timestamp']/1000)})
            path_dict[elem] = sorted(sorted_transfers,key=itemgetter('timestamp'))
            for item in path_dict[elem]:
                if item['timestamp'] not in timestamp_dict:
                    timestamp_dict[item['timestamp']] = [item["src_dst"]]
                else:
                    timestamp_dict[item['timestamp']].append(item["src_dst"])
            

        return timestamp_dict
        

In [4]:
timestamp_dict = parse_ray_csv_to_dict("../js/data/rays-v2.csv")

In [27]:
sorted(list(timestamp_dict.keys())[::3])

[11,
 12,
 13,
 19,
 22,
 24,
 26,
 29,
 30,
 31,
 35,
 42,
 47,
 48,
 49,
 55,
 60,
 63,
 66,
 70,
 73,
 75,
 76,
 81,
 86,
 88,
 89]

In [20]:
timestamp_dict_idx = 0
# print(timestamp_dict.keys())
print(timestamp_dict[12])


[[0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], [0, 37], 

In [29]:
with open("rays.js",'w') as outfile:
    outfile.write("var rays = [\n")
    ctr = 0
    for elem in sorted(list(timestamp_dict.keys())[::3]):
        outfile.write("{\n")
        outfile.write("\"timestamp\": {},\n".format(elem))
        outfile.write("\"ray_transfers\":\n")
        json.dump(timestamp_dict[elem],outfile,separators=(",",":"),indent=0)
        if ctr < len(timestamp_dict.keys()) - 1:
            outfile.write("},\n")
        else:
            outfile.write("}\n")
        ctr+= 1
    outfile.write("]\n")
        

In [ ]:
# spamwriter = csv.writer(csvfile, delimiter=' ',
#                             quotechar='|', quoting=csv.QUOTE_MINIMAL)
#     spamwriter.writerow(['Spam'] * 5 + ['Baked Beans'])
#     spamwriter.writerow(['Spam', 'Lovely Spam', 'Wonderful Spam'])